# Endoscopy generate train, validation and test sets

Undersample the healthy class, replace the polyps with adentamous and hyper-plastic examples, augment the minority classes as far as possible

In [1]:
import pandas as pd
import numpy as np

import cv2
import matplotlib.pyplot as plt
import os

from skimage import exposure
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange

Start with the ordinary kvasir-capsule established train validate and test datasets

In [2]:
img_folder = r'C:\Users\user\ML\endoscopy\kvasir-capsule'
# classes = os.listdir(img_folder)
# print(classes)

num_images_per_class = []
class_name = []

empty_images = []
empty_index = []

# if the dir has a metadata.csv file
if os.path.isfile(img_folder+'/metadata.csv'):
    df = pd.read_csv(img_folder+'/metadata.csv', sep=';')
    
df = df[['filename', 'video_id', 'frame_number', 'finding_category', 'finding_class', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']]
df.columns = ['filename', 'video_id', 'frame_number', 'category', 'label', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']

In [3]:
for i in trange(len(df)):
    df.loc[i,'filename'] = img_folder + '/' + df.loc[i,'label'] + '/' + df.loc[i, 'filename']
    
df

100%|███████████████████████████████████████████████████████████████████████████| 47248/47248 [00:57<00:00, 825.02it/s]


,filename,video_id,frame_number,category,label,x1,y1,x2,y2,x3,y3,x4,y4
0,C:\Users\user\ML\endoscopy\kvasir-capsule/Norm...,0728084c8da942d9,22803,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\Users\user\ML\endoscopy\kvasir-capsule/Norm...,0728084c8da942d9,22804,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\Users\user\ML\endoscopy\kvasir-capsule/Norm...,0728084c8da942d9,22805,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\Users\user\ML\endoscopy\kvasir-capsule/Norm...,0728084c8da942d9,22806,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\Users\user\ML\endoscopy\kvasir-capsule/Norm...,0728084c8da942d9,22807,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47243,C:\Users\user\ML\endoscopy\kvasir-capsule/Angi...,04a78ef00c5245e0,898,Luminal,Angiectasia,171.0,172.0,237.0,172.0,237.0,238.0,171.0,238.0
47244,C:\Users\user\ML\endoscopy\kvasir-capsule/Angi...,04a78ef00c5245e0,899,Luminal,Angiectasia,169.0,205.0,234.0,205.0,234.0,273.0,169.0,273.0
47245,C:\Users\user\ML\endoscopy\kvasir-capsule/Angi...,04a78ef00c5245e0,900,Luminal,Angiectasia,174.0,240.0,242.0,240.0,242.0,305.0,174.0,305.0
47246,C:\Users\user\ML\endoscopy\kvasir-capsule/Angi...,04a78ef00c5245e0,901,Luminal,Angiectasia,188.0,265.0,257.0,265.0,257.0,330.0,188.0,330.0


In [4]:
# get rid of kvasir-capsule polyps
df = df[df['label'] != 'Polyp']
df['label'].value_counts()

Normal clean mucosa     34338
Ileocecal valve          4189
Reduced Mucosal View     2906
Pylorus                  1538
Angiectasia               866
Ulcer                     854
Foreign Body              776
Lymphangiectasia          592
Erosion                   507
Blood - fresh             446
Erythema                  159
Blood - hematin            12
Ampulla of Vater           10
Name: label, dtype: int64

In [5]:
# immediately split to three sets
train, dummy = train_test_split(df, test_size=0.2, stratify=df[['label']])
validation, test = train_test_split(dummy, test_size=0.5, stratify=dummy[['label']])

print(train.shape, validation.shape, test.shape)

(37754, 13) (4719, 13) (4720, 13)


In [6]:
train['label'].value_counts()

Normal clean mucosa     27470
Ileocecal valve          3351
Reduced Mucosal View     2325
Pylorus                  1230
Angiectasia               693
Ulcer                     683
Foreign Body              621
Lymphangiectasia          473
Erosion                   406
Blood - fresh             357
Erythema                  127
Blood - hematin            10
Ampulla of Vater            8
Name: label, dtype: int64

In [7]:
validation['label'].value_counts()

Normal clean mucosa     3434
Ileocecal valve          419
Reduced Mucosal View     290
Pylorus                  154
Angiectasia               86
Ulcer                     85
Foreign Body              77
Lymphangiectasia          60
Erosion                   51
Blood - fresh             45
Erythema                  16
Blood - hematin            1
Ampulla of Vater           1
Name: label, dtype: int64

In [8]:
test['label'].value_counts()

Normal clean mucosa     3434
Ileocecal valve          419
Reduced Mucosal View     291
Pylorus                  154
Angiectasia               87
Ulcer                     86
Foreign Body              78
Lymphangiectasia          59
Erosion                   50
Blood - fresh             44
Erythema                  16
Ampulla of Vater           1
Blood - hematin            1
Name: label, dtype: int64

In [9]:
def rotate_image(img, angle, bbox):


    h, w = img.shape[:2]
    cX, cY = (w // 2, h // 2)

    # get rotation matrix (explained in section below)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)

    # get cos and sin value from the rotation matrix
    cos, sin = abs(M[0, 0]), abs(M[0, 1])

    # calculate new width and height after rotation (explained in section below)
    newW = int((h * sin) + (w * cos))
    newH = int((h * cos) + (w * sin))

    # calculate new rotation center
    M[0, 2] += (newW / 2) - cX
    M[1, 2] += (newH / 2) - cY

    # use modified rotation center and rotation matrix in the warpAffine method
    img_rotated = cv2.warpAffine(img, M, (newW, newH)) 


    # h, w = img.shape[:2] 
    # cx, cy = (int(w / 2), int(h / 2))

    bbox_tuple = [
            (bbox[0], bbox[1]),
            (bbox[2], bbox[3]),
            (bbox[4], bbox[5]),
            (bbox[6], bbox[7]),
        ] # put x and y coordinates in tuples, we will iterate through the tuples and perform rotation

    # print(bbox)
    # print(bbox[0])
    # print(bbox[0] == np.NaN)
    # print(type(bbox[0]))

    if np.isnan(np.sum(bbox)):
        bbox_rotated = [np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]
    else:
        rotated_bbox = []

        for i, coord in enumerate(bbox_tuple):

            M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
            cos, sin = abs(M[0, 0]), abs(M[0, 1])
            newW = int((h * sin) + (w * cos))
            newH = int((h * cos) + (w * sin))
            M[0, 2] += (newW / 2) - cX
            M[1, 2] += (newH / 2) - cY
            v = [coord[0], coord[1], 1]
            adjusted_coord = np.dot(M, v)
            rotated_bbox.insert(i, (adjusted_coord[0], adjusted_coord[1]))

        bbox_rotated = [int(x) for t in rotated_bbox for x in t]

        #print(bbox_rotated)

        #xmin, ymin, xmax, ymax = result[0], result[1], result[4], result[5]
        #cv2.rectangle(result, (xmin, ymin), (xmax, ymax), (0, 0, 255), thickness=2, lineType=cv2.LINE_AA)

        #plt.figure(figsize=[5,5])
        #plt.imshow(img_rotated[:,:,::-1])

    return img_rotated, bbox_rotated

In [10]:
SIZE = 336

train_images = []
train_labels = []

rotations = [90, 180, 270]

verbose = False


train = train.reset_index(drop=True)
for i in trange(len(train)):

    label = train.loc[i,'label']

    if label != 'Normal clean mucosa':

        img = cv2.imread(train.loc[i, 'filename'], cv2.IMREAD_COLOR)   
        #print(img.shape)
        bbox = list(train.iloc[i, 5:])

        #print("error - did not read properly")    
        

        width = img.shape[0]
        height = img.shape[1]

        #img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        train_images.append(img)


        # brightened
        logarithmic_corrected = exposure.adjust_log(img, 1)
        train_images.append(logarithmic_corrected)

        train = pd.concat([train, pd.DataFrame({'filename': train.loc[i, 'filename'][:-4]+'_log_c.jpg',
                                            'video_id': train.loc[i, 'video_id'],
                                            'frame_number': train.loc[i, 'frame_number'],
                                            'category': train.loc[i, 'category'],
                                            'label': train.loc[i, 'label'],
                                            'x1': bbox[0],
                                            'y1': bbox[1],
                                            'x2': bbox[2],
                                            'y2': bbox[3],
                                            'x3': bbox[4],
                                            'y3': bbox[5],
                                            'x4': bbox[6],
                                            'y4': bbox[7]}, index=[len(train)])], ignore_index=True)


        for r in rotations:

            img_rotated, bbox_rotated = rotate_image(img, r, bbox)

            #print(img_rotated.shape, bbox_rotated)
            train_images.append(img_rotated)

            train = pd.concat([train, pd.DataFrame({'filename': train.loc[i, 'filename'][:-4]+'_rot'+str(r)+'.jpg',
                                                'video_id': train.loc[i, 'video_id'],
                                                'frame_number': train.loc[i, 'frame_number'],
                                                'category': train.loc[i, 'category'],
                                                'label': train.loc[i, 'label'],
                                                'x1': bbox_rotated[0],
                                                'y1': bbox_rotated[1],
                                                'x2': bbox_rotated[2],
                                                'y2': bbox_rotated[3],
                                                'x3': bbox_rotated[4],
                                                'y3': bbox_rotated[5],
                                                'x4': bbox_rotated[6],
                                                'y4': bbox_rotated[7]}, index=[len(train)])], ignore_index=True)



100%|███████████████████████████████████████████████████████████████████████████| 37754/37754 [06:01<00:00, 104.41it/s]


In [11]:
validation_images = []
validation_labels = []

rotations = [90, 180, 270]

verbose = False


validation = validation.reset_index(drop=True)
for i in trange(len(validation)):

    label = validation.loc[i,'label']

    if label != 'Normal clean mucosa':

        img = cv2.imread(validation.loc[i, 'filename'], cv2.IMREAD_COLOR)   
        #print(img.shape)
        bbox = list(validation.iloc[i, 5:])

        #print("error - did not read properly")    
        

        width = img.shape[0]
        height = img.shape[1]

        #img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        validation_images.append(img)


        # brightened
        logarithmic_corrected = exposure.adjust_log(img, 1)
        validation_images.append(logarithmic_corrected)

        validation = pd.concat([validation, pd.DataFrame({'filename': validation.loc[i, 'filename'][:-4]+'_log_c.jpg',
                                            'video_id': validation.loc[i, 'video_id'],
                                            'frame_number': validation.loc[i, 'frame_number'],
                                            'category': validation.loc[i, 'category'],
                                            'label': validation.loc[i, 'label'],
                                            'x1': bbox[0],
                                            'y1': bbox[1],
                                            'x2': bbox[2],
                                            'y2': bbox[3],
                                            'x3': bbox[4],
                                            'y3': bbox[5],
                                            'x4': bbox[6],
                                            'y4': bbox[7]}, index=[len(validation)])], ignore_index=True)


        for r in rotations:

            img_rotated, bbox_rotated = rotate_image(img, r, bbox)

            #print(img_rotated.shape, bbox_rotated)
            validation_images.append(img_rotated)

            validation = pd.concat([validation, pd.DataFrame({'filename': validation.loc[i, 'filename'][:-4]+'_rot'+str(r)+'.jpg',
                                                'video_id': validation.loc[i, 'video_id'],
                                                'frame_number': validation.loc[i, 'frame_number'],
                                                'category': validation.loc[i, 'category'],
                                                'label': validation.loc[i, 'label'],
                                                'x1': bbox_rotated[0],
                                                'y1': bbox_rotated[1],
                                                'x2': bbox_rotated[2],
                                                'y2': bbox_rotated[3],
                                                'x3': bbox_rotated[4],
                                                'y3': bbox_rotated[5],
                                                'x4': bbox_rotated[6],
                                                'y4': bbox_rotated[7]}, index=[len(validation)])], ignore_index=True)

100%|█████████████████████████████████████████████████████████████████████████████| 4719/4719 [00:38<00:00, 124.16it/s]


In [12]:
test_images = []
test_labels = []

rotations = [90, 180, 270]

verbose = False


test = test.reset_index(drop=True)
for i in trange(len(test)):

    label = test.loc[i,'label']

    if label != 'Normal clean mucosa':

        img = cv2.imread(test.loc[i, 'filename'], cv2.IMREAD_COLOR)   
        #print(img.shape)
        bbox = list(test.iloc[i, 5:])

        #print("error - did not read properly")    
        

        width = img.shape[0]
        height = img.shape[1]

        #img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        test_images.append(img)


        # brightened
        logarithmic_corrected = exposure.adjust_log(img, 1)
        test_images.append(logarithmic_corrected)

        test = pd.concat([test, pd.DataFrame({'filename': test.loc[i, 'filename'][:-4]+'_log_c.jpg',
                                            'video_id': test.loc[i, 'video_id'],
                                            'frame_number': test.loc[i, 'frame_number'],
                                            'category': test.loc[i, 'category'],
                                            'label': test.loc[i, 'label'],
                                            'x1': bbox[0],
                                            'y1': bbox[1],
                                            'x2': bbox[2],
                                            'y2': bbox[3],
                                            'x3': bbox[4],
                                            'y3': bbox[5],
                                            'x4': bbox[6],
                                            'y4': bbox[7]}, index=[len(test)])], ignore_index=True)


        for r in rotations:

            img_rotated, bbox_rotated = rotate_image(img, r, bbox)

            #print(img_rotated.shape, bbox_rotated)
            test_images.append(img_rotated)

            test = pd.concat([test, pd.DataFrame({'filename': test.loc[i, 'filename'][:-4]+'_rot'+str(r)+'.jpg',
                                                'video_id': test.loc[i, 'video_id'],
                                                'frame_number': test.loc[i, 'frame_number'],
                                                'category': test.loc[i, 'category'],
                                                'label': test.loc[i, 'label'],
                                                'x1': bbox_rotated[0],
                                                'y1': bbox_rotated[1],
                                                'x2': bbox_rotated[2],
                                                'y2': bbox_rotated[3],
                                                'x3': bbox_rotated[4],
                                                'y3': bbox_rotated[5],
                                                'x4': bbox_rotated[6],
                                                'y4': bbox_rotated[7]}, index=[len(test)])], ignore_index=True)

100%|█████████████████████████████████████████████████████████████████████████████| 4720/4720 [00:39<00:00, 118.34it/s]


In [13]:
train

,filename,video_id,frame_number,category,label,x1,y1,x2,y2,x3,y3,x4,y4
0,C:\Users\user\ML\endoscopy\kvasir-capsule/Ileo...,131368cc17e44240,31685,Anatomy,Ileocecal valve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\Users\user\ML\endoscopy\kvasir-capsule/Redu...,8885668afb844852,25691,Luminal,Reduced Mucosal View,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\Users\user\ML\endoscopy\kvasir-capsule/Norm...,dac1e27f7e4d4ef5,14990,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\Users\user\ML\endoscopy\kvasir-capsule/Norm...,64440803f87b4843,7227,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\Users\user\ML\endoscopy\kvasir-capsule/Norm...,131368cc17e44240,5622,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78885,C:\Users\user\ML\endoscopy\kvasir-capsule/Ulce...,7a47e8eacea04e64,52283,Luminal,Ulcer,334.0,126.0,333.0,258.0,244.0,258.0,244.0,126.0
78886,C:\Users\user\ML\endoscopy\kvasir-capsule/Ileo...,5bb1d3cc7dc64cec,34129,Anatomy,Ileocecal valve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78887,C:\Users\user\ML\endoscopy\kvasir-capsule/Ileo...,5bb1d3cc7dc64cec,34129,Anatomy,Ileocecal valve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78888,C:\Users\user\ML\endoscopy\kvasir-capsule/Ileo...,5bb1d3cc7dc64cec,34129,Anatomy,Ileocecal valve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
print(train['label'].value_counts())
print(validation['label'].value_counts())
print(test['label'].value_counts())

Normal clean mucosa     27470
Ileocecal valve         16755
Reduced Mucosal View    11625
Pylorus                  6150
Angiectasia              3465
Ulcer                    3415
Foreign Body             3105
Lymphangiectasia         2365
Erosion                  2030
Blood - fresh            1785
Erythema                  635
Blood - hematin            50
Ampulla of Vater           40
Name: label, dtype: int64
Normal clean mucosa     3434
Ileocecal valve         2095
Reduced Mucosal View    1450
Pylorus                  770
Angiectasia              430
Ulcer                    425
Foreign Body             385
Lymphangiectasia         300
Erosion                  255
Blood - fresh            225
Erythema                  80
Blood - hematin            5
Ampulla of Vater           5
Name: label, dtype: int64
Normal clean mucosa     3434
Ileocecal valve         2095
Reduced Mucosal View    1455
Pylorus                  770
Angiectasia              435
Ulcer                    430
Foreign

Now add polyp data - polyps are NOT being augmented as there are more than enough

In [15]:
polyps_folder = r'C:\Users\user\ML\endoscopy\polyps'

polyps_df = pd.read_csv(polyps_folder+'/combined_polyps_df.csv')

polyps_df['video_id'] = "abcd"
polyps_df['frame_number'] = -1
polyps_df['category'] = "Luminal"

polyps_df = polyps_df[['image_filename', 'video_id', 'frame_number', 'category', 'label', 'height', 'width', 'xmax', 'xmin', 'ymax', 'ymin']]
polyps_df.columns = ['filename', 'video_id', 'frame_number', 'category', 'label', 'height', 'width', 'xmax', 'xmin', 'ymax', 'ymin']

polyps_df = polyps_df[polyps_df['label'] != 'missing']
polyps_df = polyps_df.reset_index(drop=True)
polyps_df['label'].value_counts()


adenomatous     19240
hyperplastic    16741
Name: label, dtype: int64

In [16]:
polyps_df['x1'] = 0
polyps_df['y1'] = 0
polyps_df['x2'] = 0
polyps_df['y2'] = 0
polyps_df['x3'] = 0
polyps_df['y3'] = 0
polyps_df['x4'] = 0
polyps_df['y4'] = 0

polyps_df = polyps_df.reset_index(drop=True)

for i in trange(len(polyps_df)):
    polyps_df.loc[i, 'x1'] = polyps_df.loc[i, 'xmin']
    polyps_df.loc[i, 'y1'] = polyps_df.loc[i, 'ymin']
    polyps_df.loc[i, 'x2'] = polyps_df.loc[i, 'xmax']
    polyps_df.loc[i, 'y2'] = polyps_df.loc[i, 'ymin']
    polyps_df.loc[i, 'x3'] = polyps_df.loc[i, 'xmax']
    polyps_df.loc[i, 'y3'] = polyps_df.loc[i, 'ymax']
    polyps_df.loc[i, 'x4'] = polyps_df.loc[i, 'xmin']
    polyps_df.loc[i, 'y4'] = polyps_df.loc[i, 'ymax']

polyps_df

100%|████████████████████████████████████████████████████████████████████████████| 35981/35981 [11:43<00:00, 51.15it/s]


,filename,video_id,frame_number,category,label,height,width,xmax,xmin,ymax,ymin,x1,y1,x2,y2,x3,y3,x4,y4
0,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,288,384,343,164,279,113,164,113,343,113,343,279,164,279
1,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,hyperplastic,464,592,357,261,294,196,261,196,357,196,357,294,261,294
2,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,576,768,223,102,497,353,102,353,223,353,223,497,102,497
3,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,hyperplastic,464,592,438,398,321,249,398,249,438,249,438,321,398,321
4,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,464,592,153,46,297,128,46,128,153,128,153,297,46,297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35976,C:\Users\user\ML\endoscopy\polyps\test2019\Ima...,abcd,-1,Luminal,hyperplastic,576,768,472,405,284,207,405,207,472,207,472,284,405,284
35977,C:\Users\user\ML\endoscopy\polyps\test2019\Ima...,abcd,-1,Luminal,hyperplastic,576,768,467,394,278,207,394,207,467,207,467,278,394,278
35978,C:\Users\user\ML\endoscopy\polyps\test2019\Ima...,abcd,-1,Luminal,hyperplastic,576,768,445,372,269,198,372,198,445,198,445,269,372,269
35979,C:\Users\user\ML\endoscopy\polyps\test2019\Ima...,abcd,-1,Luminal,hyperplastic,576,768,434,361,260,189,361,189,434,189,434,260,361,260


In [17]:
polyps_df = polyps_df.drop(['height', 'width', 'xmax', 'xmin', 'ymax', 'ymin'], axis=1)
polyps_df

,filename,video_id,frame_number,category,label,x1,y1,x2,y2,x3,y3,x4,y4
0,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,164,113,343,113,343,279,164,279
1,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,hyperplastic,261,196,357,196,357,294,261,294
2,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,102,353,223,353,223,497,102,497
3,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,hyperplastic,398,249,438,249,438,321,398,321
4,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,46,128,153,128,153,297,46,297
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35976,C:\Users\user\ML\endoscopy\polyps\test2019\Ima...,abcd,-1,Luminal,hyperplastic,405,207,472,207,472,284,405,284
35977,C:\Users\user\ML\endoscopy\polyps\test2019\Ima...,abcd,-1,Luminal,hyperplastic,394,207,467,207,467,278,394,278
35978,C:\Users\user\ML\endoscopy\polyps\test2019\Ima...,abcd,-1,Luminal,hyperplastic,372,198,445,198,445,269,372,269
35979,C:\Users\user\ML\endoscopy\polyps\test2019\Ima...,abcd,-1,Luminal,hyperplastic,361,189,434,189,434,260,361,260


In [18]:
polyps_train, polyps_dummy = train_test_split(polyps_df, test_size=0.2, stratify=polyps_df[['label']])
polyps_validation, polyps_test = train_test_split(polyps_dummy, test_size=0.5, stratify=polyps_dummy[['label']])

print(polyps_train['label'].value_counts())
print(polyps_validation['label'].value_counts())
print(polyps_test['label'].value_counts())

adenomatous     15392
hyperplastic    13392
Name: label, dtype: int64
adenomatous     1924
hyperplastic    1674
Name: label, dtype: int64
adenomatous     1924
hyperplastic    1675
Name: label, dtype: int64


In [19]:
polyps_train

,filename,video_id,frame_number,category,label,x1,y1,x2,y2,x3,y3,x4,y4
26087,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,284,7,347,7,347,110,284,110
17363,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,276,216,395,216,395,328,276,328
25499,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,hyperplastic,283,131,401,131,401,230,283,230
32575,C:\Users\user\ML\endoscopy\polyps\test2019\Ima...,abcd,-1,Luminal,hyperplastic,308,199,433,199,433,286,308,286
23364,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,hyperplastic,179,100,292,100,292,222,179,222
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18138,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,223,161,344,161,344,285,223,285
14534,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,hyperplastic,222,205,386,205,386,323,222,323
10973,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,296,168,354,168,354,276,296,276
29692,C:\Users\user\ML\endoscopy\polyps\val2019\Imag...,abcd,-1,Luminal,adenomatous,342,175,482,175,482,401,342,401


Now write to the training data

In [20]:
train = pd.concat([train, polyps_train])
validation = pd.concat([validation, polyps_validation])
test = pd.concat([test, polyps_test])

In [21]:
train

,filename,video_id,frame_number,category,label,x1,y1,x2,y2,x3,y3,x4,y4
0,C:\Users\user\ML\endoscopy\kvasir-capsule/Ileo...,131368cc17e44240,31685,Anatomy,Ileocecal valve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\Users\user\ML\endoscopy\kvasir-capsule/Redu...,8885668afb844852,25691,Luminal,Reduced Mucosal View,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\Users\user\ML\endoscopy\kvasir-capsule/Norm...,dac1e27f7e4d4ef5,14990,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\Users\user\ML\endoscopy\kvasir-capsule/Norm...,64440803f87b4843,7227,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\Users\user\ML\endoscopy\kvasir-capsule/Norm...,131368cc17e44240,5622,Luminal,Normal clean mucosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18138,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,223.0,161.0,344.0,161.0,344.0,285.0,223.0,285.0
14534,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,hyperplastic,222.0,205.0,386.0,205.0,386.0,323.0,222.0,323.0
10973,C:\Users\user\ML\endoscopy\polyps\train2019\Im...,abcd,-1,Luminal,adenomatous,296.0,168.0,354.0,168.0,354.0,276.0,296.0,276.0
29692,C:\Users\user\ML\endoscopy\polyps\val2019\Imag...,abcd,-1,Luminal,adenomatous,342.0,175.0,482.0,175.0,482.0,401.0,342.0,401.0


In [22]:
train.to_csv('C:/Users/user/ML/endoscopy/kvasir-capsule-augments/train_augment_polyp.csv', index=False)

# for i in trange(len(train_images)):
#     train_images[i] = cv2.cvtColor(train_images[i], cv2.COLOR_RGB2BGR)
#     cv2.imwrite('C:/Users/user/ML/endoscopy/kvasir-capsule-augments/train_augment_polyp'+train.loc[i,'filename'], train_images[i])

PermissionError: [Errno 13] Permission denied: 'C:/Users/user/ML/endoscopy/kvasir-capsule-augments/train_augment_polyp.csv'

In [ ]:
validation.to_csv('C:/Users/user/ML/endoscopy/kvasir-capsule-augments/validation_augment_polyp.csv', index=False)

# for i in trange(len(validate_images)):
#     validate_images[i] = cv2.cvtColor(train_images[i], cv2.COLOR_RGB2BGR)
#     cv2.imwrite('C:/Users/user/ML/endoscopy/kvasir-capsule-augments/validate_augment_polyp'+validate.loc[i,'filename'], validate_images[i])

In [ ]:
test.to_csv('C:/Users/user/ML/endoscopy/kvasir-capsule-augments/test_augment_polyp.csv', index=False)

# for i in trange(len(test_images)):
#     test_images[i] = cv2.cvtColor(train_images[i], cv2.COLOR_RGB2BGR)
#     cv2.imwrite('C:/Users/user/ML/endoscopy/kvasir-capsule-augments/test_augment_polyp'+train.loc[i,'filename'], train_images[i])

In [ ]:
# plt.imshow(train_images[1000])
# plt.show()

In [ ]:
train['label'].value_counts()

In [ ]:
validation['label'].value_counts()